# Exercise 4

## 1. Constructing the matrix X

In the first step we are going to implemnt the function X = construct_X(M, alphas, Np = None) without any sparse matrix optimizations

In [1]:
import numpy as np

In [2]:
def construct_X(M, alphas, Np = None):
    """
    Construct a matrix X which describes the setup of a simplyfied
    CT scanner after the formular X*beta = y, with beta being a flattend 
    tomograph and y being a flattened sinogram.
    
    M = Disired tomograph size D = M x M
    alpha = list of measurement angles
    Np = optional sensor resolution
    """

    D = M*M
    N_o = len(alphas)

    # If no resolution is givin a value of sqrt(2)*M is set
    if Np == None:
        Np = int(np.ceil(np.sqrt(2) * M))

    X = np.zeros((Np*N_o, D))
    
    for i_o, alpha in enumerate(alphas):
        # Rotate coordinate
        angle = np.deg2rad(alpha)
        
        for j_a in range(M):
            for j_b in range(M):
                j = j_a + M * j_b

                # Set center as coordinate (0,0)
                x = j_a - (M - 1) / 2
                y = j_b - (M - 1) / 2

                # Rotate sensor 
                x_rot =  x * np.cos(angle) - y * np.sin(angle)

                # Project the tomogram's pixel coordinates on sensor array
                sensor_location = (x_rot + (Np - 1) / 2)
                ray_intensity = sensor_location - np.floor(sensor_location)
                
                # Determine the sensor indices influenced by the ray
                sensor_index1 = int(np.floor(sensor_location))
                sensor_index2 = int(np.ceil(sensor_location))
                
                # Calculate the weights based on the distance from the center of the bins
                if sensor_index1 >= 0 and sensor_index1 < Np:
                    X[sensor_index1 + Np * i_o, j] += 1 - ray_intensity
                if sensor_index2 >= 0 and sensor_index2 < Np:    
                    X[sensor_index2 + Np * i_o, j] += ray_intensity

    return X


## 2. Recovering the image

## 3. Minimizing the radiation dose